In [19]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


In [20]:
# Define a function to keep certain words in English
def keep_english_words(translation):
    english_words = ['demo', 'headset', 'feedback', 'comment', 'section',
                     'video', 'products', 'bag']
    for word in english_words:
        translation = translation.replace(word, f'[{word}]')
    return translation


In [21]:
# Define a function to translate English to Hinglish
def translate_to_hinglish(input_text):
    try:
        input_tokens = input_text.split()
        output_tokens = []

        for token in input_tokens:
            if token.lower() in english_words:
                output_tokens.append(f"[{token}]")
            else:
                output_tokens.append(translation_mapping.get(token.lower(), token))

        hinglish_translation = ' '.join(output_tokens)
        return hinglish_translation
    except Exception as e:
        return str(e)

In [22]:
# Sample English sentences
english_sentences = [
    "Definitely share your feedback in the comment section.",
    "So even if it's a big video, I will clearly mention all the products.",
    "I was waiting for my bag."
]

# Create a set of English words to keep
english_words = {'demo', 'headset', 'feedback', 'comment', 'section',
                 'video', 'products', 'bag'}

# Mapping of English to Hinglish for non-English words
translation_mapping = {
    "definitely": "ज़रूर",
    "share": "साझा",
    "your": "तुम्हारी",
    "in": "में",
    "the": "यह",
    "so": "तो",
    "even": "यहाँ तक कि",
    "it's": "यह है",
    "a": "एक",
    "big": "बड़ा",
    "I": "मैं",
    "will": "मैं",
    "clearly": "स्पष्ट रूप से",
    "mention": "उल्लेख",
    "all": "सभी",
    "the": "यह",
    "I": "मैं",
    "was": "था",
    "waiting": "इंतजार",
    "for": "के लिए",
    "my": "मेरी",
    "bag": "बैग"
}

# Define the inverse translation mapping
translation_mapping_inv = {v: k for k, v in translation_mapping.items()}

In [23]:
# Define the inverse translation mapping
translation_mapping_inv = {v: k for k, v in translation_mapping.items()}

# Combine translation and Seq2Seq model

# Define a simple sequence-to-sequence model
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq):
        embedded = self.encoder(input_seq)
        output_seq = self.decoder(embedded)
        return output_seq

# Dummy data for training
input_size = 50  # Vocabulary size for English
hidden_size = 128
output_size = 50  # Vocabulary size for Hinglish

In [24]:
# Create model
model = Seq2Seq(input_size, hidden_size, output_size)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy input sequence
input_seq = torch.tensor([1, 2, 3, 4, 5])

In [25]:
# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    output_seq = model(input_seq)

    # Dummy target sequence
    target_seq = torch.tensor([6, 7, 8, 9, 10])

    loss = criterion(output_seq, target_seq)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

# Inference (translating English to Hinglish)
input_seq = torch.tensor([1,2,3,4,5])
output_seq = model(input_seq)
output_indices = torch.argmax(output_seq, dim=1)
predicted_hinglish_sentence = ' '.join([translation_mapping_inv.get(idx.item(), str(idx.item())) for idx in output_indices])

# Print results
print("English Sentence:")
print(english_sentences)  # Print the first English sentence
print(predicted_hinglish_sentence)  # Print the predicted Hinglish sentence

# Translate and print Hinglish sentences for the rest of the English sentences
for i in range(1, len(english_sentences)):
    hinglish_translation = translate_to_hinglish(english_sentences[i])
    hinglish_translation = keep_english_words(hinglish_translation)
    print(f"\nEnglish: {english_sentences[i]}")
    print(f"Hinglish: {hinglish_translation}")


Epoch [100/100], Loss: 0.0398
English Sentence:
['Definitely share your feedback in the comment section.', "So even if it's a big video, I will clearly mention all the products.", 'I was waiting for my bag.']
6 7 8 9 10

English: So even if it's a big video, I will clearly mention all the products.
Hinglish: तो यहाँ तक कि if यह है एक बड़ा [video], I मैं स्पष्ट रूप से उल्लेख सभी यह [products].

English: I was waiting for my bag.
Hinglish: I था इंतजार के लिए मेरी [bag].
